In [233]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sys
np.set_printoptions(threshold=sys.maxsize)


# Preprocessing data

In [49]:
df=pd.read_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-1/nifty_sensex_close_data/M&M.NS_2011_to_2023.csv')

In [197]:
df_close=df['M&M.NS_close']
df_close
df_close = df_close.astype(float)
df_close.shape

n=615 #number of data points
w=124 #window size
#training array
trainval_arr_inp=np.zeros([n-w,w]) #input array

#filling the input array
for i in range (n-w):
    for j in range(w):
        trainval_arr_inp[i][j]=df_close[i+j]

#output array
trainval_arr_out=np.array([df_close[i+w] for i in range(n-w)])
trainval_arr_out=trainval_arr_out.reshape(n-w,1)
trainval_arr_out.shape

trainval_arr=np.column_stack((trainval_arr_inp,trainval_arr_out)) #training array

#shuffle the rows
np.random.seed(0)
np.random.shuffle(trainval_arr)


In [198]:
train_arr=trainval_arr[:int(0.7*(n-w)),:] #training array
val_arr=trainval_arr[int(0.7*(n-w)):,:] #validation array

X_train=train_arr[:,:-1]
y_train=train_arr[:,-1]

X_val=val_arr[:,:-1]
y_val=val_arr[:,-1]

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)


(343, 124)
(343,)
(148, 124)
(148,)


In [226]:
testadata = np.zeros(w)
for i in range(w):
    testadata[i] = df_close[n+i]
testdata = testadata.reshape(1, w)
print(testdata.shape)

df_close[n+w]

(1, 124)


435.1484375

# model 

In [199]:
rf=RandomForestRegressor(n_estimators=1000,random_state=0,verbose=1)
rf.fit(X_train,y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.7s finished


RandomForestRegressor(n_estimators=1000, random_state=0, verbose=1)

In [200]:
y_pred=rf.predict(X_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


In [229]:
#error and accuracy for validation data

#mae
mae=np.mean(abs(y_val-y_pred))
print('mae:',mae)

#accuracy
accuracy=100*(1-mae/np.mean(y_val))
print('accuracy:',accuracy)

#r2 score
from sklearn.metrics import r2_score
r2=r2_score(y_val,y_pred)
print('r2:',r2)

mae: 4.885468611433694
accuracy: 98.64674520811865
r2: 0.9818350091438356


In [230]:
y_tpred=rf.predict(testdata)
print('predicted:',y_tpred)

predicted: [435.08623758]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished


In [232]:
#error and accuracy for test data for 1 day

#mae
mae_t=np.mean(abs(df_close[n+w]-y_tpred))
print('mae:',mae_t)

#accuracy
accuracy_t=100*(1-mae_t/df_close[n+w])  
print('accuracy:',accuracy_t)



mae: 0.062199920654279595
accuracy: 99.98570604527237


In [234]:
#grid search for hyperparameter tuning

param_grid = {'n_estimators': [100, 200, 300, 1000], 
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

In [235]:
rf_grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

In [236]:
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 

[CV] END .....................max_depth=10, n_estimators=100; total time=   1.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=   1.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=   1.4s
[CV] END .....................max_depth=10, n_estimators=100; total time=   1.4s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


[CV] END .....................max_depth=10, n_estimators=100; total time=   1.3s


...

[CV] END ...................max_depth=100, n_estimators=1000; total time=  12.4s
[CV] END ...................max_depth=100, n_estimators=1000; total time=  12.7s
[CV] END ...................max_depth=100, n_estimators=1000; total time=  12.5s


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   12.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   12.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ...................max_depth=100, n_estimators=1000; total time=  12.4s
[CV] END ...................max_depth=100, n_estimators=1000; total time=  12.3s


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=1000, random_state=0,
                                             verbose=1),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [237]:
rf_grid.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [238]:
rf_grid.best_score_

0.9788520430725347

In [240]:
rfn=RandomForestRegressor(n_estimators=100,max_depth=10,random_state=0,verbose=1)
rfn.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished


RandomForestRegressor(max_depth=10, random_state=0, verbose=1)

In [241]:
y_predn=rfn.predict(X_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [242]:
#error and accuracy for validation data

#mae
maen=np.mean(abs(y_val-y_predn))
print('mae:',maen)

#accuracy
accuracyn=100*(1-maen/np.mean(y_val))
print('accuracy:',accuracyn)

#r2 score
r2n=r2_score(y_val,y_predn)
print('r2:',r2n)

mae: 4.824177981059165
accuracy: 98.66372246165326
r2: 0.9822659916079624


In [243]:
#error and accuracy for test data for 1 day

y_tpredn=rfn.predict(testdata)
print('predicted:',y_tpredn)

#mae
mae_tn=np.mean(abs(df_close[n+w]-y_tpredn))
print('mae:',mae_tn)

#accuracy
accuracy_tn=100*(1-mae_tn/df_close[n+w])
print('accuracy:',accuracy_tn)

predicted: [434.50334901]
mae: 0.6450884873361815
accuracy: 99.85175438270161


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
